In [1]:
from openai import OpenAI
import json

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [4]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-mpnet-base-v2')

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
len(model.encode('My name is Alex.'))

768

In [6]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
operations = []

for doc in documents:
    doc['text_vector'] = model.encode(doc['text']).tolist()
    operations.append(doc)

In [8]:
operations[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp',
 'text_vector': [-0.04103037342429161,
  0.02583414688706398,
  -0.03680191561579704,
  -0.020898308604955673,
  -0.020596222952008247,
  0.009353749454021454,
  -0.003331621875986457,
  -0.009491918608546257,
  0.030117960646748543,
  0.019082099199295044,
  0.012690030038356781,
  -0.017078785225749016,
  -0.0016323721501976252,
  0.12997256219387054,
  0.030969271436333656,
  -0.025823596864938736,
  0.027823062613606453,
  0.025159751996397972,
  -0.08081215620040894,
  -0.0036174245178699493,
  -0.008902010507881641,
  0.0034048687666654587,
  -0.023009341210126877,
  -0.034045297652482986,
  0.024598613381385803,
  0.013545605354011059,
  -0.025438982993364334,
  0.011951032094657421,
  -0.020540105178952217,
  -0.010077491402626038,
  0.02057531848549

In [9]:
from elasticsearch import Elasticsearch

In [10]:
es_client = Elasticsearch('http://localhost:9200')

In [11]:
es_client.info()

ObjectApiResponse({'name': 'ed7f34f7ba70', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'ITxmhXLfR5G1xMxQyd0o6Q', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [12]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "text_vector": {"type" : "dense_vector", "dims":768, "index" : True, "similarity" : "cosine"}
        }
    }
}

In [13]:
index_name = 'course-questions'

In [14]:
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body = index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [15]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [16]:
q = 'windows or mac?'
vector_search = model.encode(q)

In [17]:
query = {
    "field" : "text_vector",
    "query_vector": vector_search,
    "k" : 3,
    "num_candidates" : 10000
}

In [18]:
res = es_client.search(index = index_name, knn = query, source = ["text", "section","question", "course"])

In [19]:
res['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'eoesupABMC8uee5BYikT',
  '_score': 0.71479183,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course-questions',
  '_id': 'ToesupABMC8uee5BaSvV',
  '_score': 0.60555583,
  '_source': {'question': "The answer I get for one of the homework questions doesn't match any of the options. What should I do?",
   'course': 'machine-learning-zoomcamp',
   'section': '2. Machine Learning for Regression',
   'text': 'That’s normal. We all have different environments: our computers have different versions of OS and different versions of libraries — even different versions of Python.\nIf it’s the case, just select the option that’s closest to your answer'}},
 {'_index': 'course-questions',
  '

In [20]:
knn_query = {
    "field" : "text_vector",
    "query_vector" : vector_search,
    "k" : 5,
    "num_candidates" : 10000
}

In [21]:
advanced_response = es_client.search(
    index=index_name,
    query={
        "match": {"course": "data-engineering-zoomcamp"},
    },
    knn=knn_query,
    size=5,
    explain=True
)

In [22]:
advanced_response['hits']['hits']

[{'_shard': '[course-questions][0]',
  '_node': '_VTaGQOURCCApoEAT2VIUw',
  '_index': 'course-questions',
  '_id': 'eoesupABMC8uee5BYikT',
  '_score': 1.179327,
  '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
   'section': 'General course-related questions',
   'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'text_vector': [-0.026965495198965073,
    -0.0006259257206693292,
    -0.016629504039883614,
    0.05285143107175827,
    0.05476531386375427,
    -0.03133983165025711,
    0.029942642897367477,
    -0.04808562621474266,
    0.04467548429965973,
    0.005839455407112837,
    0.016233131289482117,
    0.012001212686300278,
    -0.0312223918735981,
    0.01660064235329628,
    -0.04886900633573532,
    -0.06496303528547287,
    0.046434178948402405,
    -0.009297648444771767,
    -0.0642528235912323,
    -0.01373270619660616,
  